In [61]:
import pandas as pd
import os

df1 = pd.read_csv('csv/20240422-20240620.csv', encoding='shift-jis')
df2 = pd.read_csv('csv/20240621-20240820.csv', encoding='shift-jis')
df3 = pd.read_csv('csv/20240821-20241020.csv', encoding='shift-jis')
df4 = pd.read_csv('csv/20241021-20241220.csv', encoding='shift-jis')
df5 = pd.read_csv('csv/20241221-20250219.csv', encoding='shift-jis')
df6 = pd.read_csv('csv/20250220-20250331.csv', encoding='shift-jis')
df7 = pd.read_csv('csv/20250401-20250501.csv', encoding='shift-jis')

df1

columns_to_keep = ["注文日","商品名","請求金額", "合計金額","個数","送料合計","ポイント利用額","クーポン利用総額"]

df1 = df1[columns_to_keep]
df2 = df2[columns_to_keep]
df3 = df3[columns_to_keep]
df4 = df4[columns_to_keep]
df5 = df5[columns_to_keep]
df6 = df6[columns_to_keep]
df7 = df7[columns_to_keep]  

merged_df = pd.concat([df1, df2, df3, df4, df5, df6, df7], ignore_index=True)
merged_df = merged_df.fillna(0)

merged_df = merged_df[~merged_df["商品名"].str.contains("レキシオネオ|LexioNeo|ボードゲーム|日焼け対策|ひやけ|毛穴ケア", na=False)]

# インデックスを振りなおす
merged_df = merged_df.reset_index(drop=True)
merged_df


,注文日,商品名,請求金額,合計金額,個数,送料合計,ポイント利用額,クーポン利用総額
0,2024-04-26,美品【中古】ピエールカルダン pierre cardin お洒落ジャケット 表記Mサイズ(9...,23171,25600,1,0,429.0,2000
1,2024-04-26,美品【中古】ポールスミス Poul Smith テーラードジャケット ジャケット 表記 38...,23171,25600,1,0,429.0,2000
2,2024-04-26,新品同様【中古】ホコモモラ Jocomomola 緑 涼感 素敵ブラウス トップス 42号/...,9800,10800,1,0,0.0,1000
3,2024-04-27,美品【中古】組曲 KUMIKYOKU ワンピース 異素材ワンピース 大きいサイズ 表記 5号...,21900,21900,1,0,0.0,0
4,2024-05-02,美品【中古】ミラオーウェン mila owen タイトスカート ニットスカート スカート レ...,6300,16300,1,0,10000.0,0
...,...,...,...,...,...,...,...,...
400,2025-03-27,美品【中古】ジブリーヌ Zibeline 東京イギン フォーマルジャケット ノーカラージャケ...,33500,33500,1,0,0.0,0
401,2025-03-29,新品同様【中古】スキャパ SCAPA フレアスカート 表記 38号 9号 M 相当 古着 紺...,8400,8900,1,0,500.0,0
402,2025-04-03,美品【中古】レオナール LEONARD デニムスカート 膝丈スカート スカート レディース ...,28000,28780,1,0,780.0,0
403,2025-04-08,新品同様【中古】エプリス EPRISE NOIR3 東京ブラウス フォーマルブラウス ブラウ...,18000,20000,1,0,0.0,2000


In [62]:
# csvファイル保存
merged_df.to_csv('merged_20240422-2025-0412.csv', index=False, encoding='utf-8-sig')

In [78]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Wikipedia URL
url = 'https://ja.wikipedia.org/wiki/ファッションブランド一覧'

# HTML取得
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# 指定クラスのdivを取得
content_div = soup.find('div', class_='mw-content-ltr mw-parser-output')

# その中にある <ul> をすべて抽出
brand_names = []
if content_div:
    for ul in content_div.find_all('ul'):
        for li in ul.find_all('li'):
            text = li.get_text().strip()
            if text:
                brand_names.append(text)

# 重複を除去
brand_names = list(set(brand_names))

# ブランド名リストをDataFrameに変換
df_brands = pd.DataFrame(brand_names, columns=["ブランド名"])

# CSVとして保存
df_brands.to_csv("brand_list.csv", index=False, encoding="utf-8-sig")

In [80]:
import pandas as pd

# CSVの読み込み（あなたの既存のbrand_list_cleaned.csvを前提）
df = pd.read_csv("brand_list_cleaned.csv", encoding="utf-8-sig")

# スラッシュで分割し、前後をstripして英語・日本語に振り分け
def split_brand_name(name):
    parts = [p.strip() for p in name.split('/')]

    if len(parts) == 2:
        return pd.Series({'ブランド名（英語）': parts[0], 'ブランド名（日本語）': parts[1]})
    elif re.match(r'^[A-Za-z0-9\s\-]+$', name):
        # 英語名のみ
        return pd.Series({'ブランド名（英語）': name, 'ブランド名（日本語）': ''})
    else:
        # 日本語名のみまたはその他
        return pd.Series({'ブランド名（英語）': '', 'ブランド名（日本語）': name})

# 新しい列に分割した結果を追加
split_df = df['ブランド名'].apply(split_brand_name)

# 元のDataFrameと結合（もしくはsplit_dfだけ保存でもOK）
new_df = pd.concat([split_df], axis=1)

# 保存
new_df.to_csv("brand_list_separated.csv", index=False, encoding="utf-8-sig")

# サンプル表示
print(new_df.head(20))


                         ブランド名（英語）                ブランド名（日本語）
0                               ±0                 プラスマイナスゼロ
1                           + KBF+                 ケービーエフプラス
2              0000 by n゜44 A.C.C.    ゼロゼロゼロゼロバイナンバー44エーシーシー
3                       0044 paris               ゼロゼロヨンヨン・パリ
4                         02DERIV.                   ツーディライブ
5                             0941                オーナインフォーワン
6                   1 to 10 people                ワントゥテンピープル
7                           10Deep                    テンディープ
8                          15jyugo                      ジューゴ
9                   18th Amendment           エイティーンス・アメンドメント
10                            1984            ナインティーンエイティフォー
11                     1LDK SELECT            ワンエルディーケー・セレクト
12                       1 Percent                   イチパーセント
13              20000000 fragments        トゥエンティーミリオンフラッグメンツ
14              21_21 DESIGN SIGHT       トゥーワン・トゥーワン・デザインサイト
15                      

In [86]:
import pandas as pd
import re

# ブランド名一覧（英語・日本語）
brand_df = pd.read_csv("brand_list_separated.csv").dropna()
brand_df["ブランド名（英語）"] = brand_df["ブランド名（英語）"].astype(str).str.strip()
brand_df["ブランド名（日本語）"] = brand_df["ブランド名（日本語）"].astype(str).str.strip()

# ブランド名（英語・日本語）を1つのリストにまとめ、重複を除き、長さ順にソート（長い順）
brand_names = list(set(brand_df["ブランド名（英語）"].tolist() + brand_df["ブランド名（日本語）"].tolist()))
brand_names = sorted(brand_names, key=lambda x: len(x), reverse=True)

# 商品データ
products_df = pd.read_csv("merged_20240422-2025-0412.csv")

# ブランド名抽出関数
def extract_brand(product_name):
    for brand in brand_names:
        brand_clean = re.escape(brand)

        if re.search(rf"\b{brand_clean}\b", product_name):  # 英語：単語境界で完全一致
            return brand
        elif re.search(rf"{brand_clean}(?!\w)", product_name):  # 日本語：後ろに続く文字が英数字でない
            return brand
    return None

# 抽出処理
products_df["抽出ブランド名"] = products_df["商品名"].astype(str).apply(extract_brand)

# 結果表示（先頭5行）
print(products_df[["商品名", "抽出ブランド名"]].head())


                                                 商品名     抽出ブランド名
0  美品【中古】ピエールカルダン pierre cardin お洒落ジャケット 表記Mサイズ(9...        None
1  美品【中古】ポールスミス Poul Smith テーラードジャケット ジャケット 表記 38...        None
2  新品同様【中古】ホコモモラ Jocomomola 緑 涼感 素敵ブラウス トップス 42号/...  Jocomomola
3  美品【中古】組曲 KUMIKYOKU ワンピース 異素材ワンピース 大きいサイズ 表記 5号...         カラー
4  美品【中古】ミラオーウェン mila owen タイトスカート ニットスカート スカート レ...         ila


In [88]:
import pandas as pd

# 商品名が含まれるCSVファイルを読み込み
df = pd.read_csv('merged_20240422-2025-0412.csv')

# 商品名列名（例： '商品名' など）に応じて適宜修正
column_name = '商品名'

# 空白や全角記号「】」で分割
split_df = df[column_name].fillna('').apply(lambda x: pd.Series(re.split(r'[ 】]+', x.strip())))

# 分割された単語だけをCSVとして保存
split_df.to_csv('商品名_単語分割のみ.csv', index=False, encoding='utf-8-sig')

In [109]:
import pandas as pd

# CSV読み込み
df = pd.read_csv('csv/merged_20240422-2025-0412.csv')

# 商品名列（適宜変更）
column_name = '商品名'

# ラベル付け関数
def label_season(text):
    if pd.isna(text):
        return ''
    text = str(text).lower()

    if any(kw in text for kw in ['春 夏 秋 冬','オールシーズン']):
        return '通年'
    elif any(kw in text for kw in ['秋冬', "秋 冬"]):
        return '秋冬'
    elif any(kw in text for kw in ['春秋', '春 秋']):
        return '春秋'
    elif any(kw in text for kw in ['春夏', "春 夏"]):
        return '春夏'
    return ''  # 判別できなければ空欄

# 新しい列にラベルを付ける
df['季節ラベル'] = df[column_name].apply(label_season)

# 結果をCSVに保存
df.to_csv('商品名_季節ラベル付き.csv', index=False, encoding='utf-8-sig')


In [115]:
df = pd.read_csv('csv/商品名_単語分割のみ.csv', encoding='utf-8-sig')

# Select the second column (column index 1)
df = df.iloc[:, 1]

df.to_csv('csv/brand_20240422-2025-0412.csv', index=False, encoding='utf-8-sig')

In [ ]:
import pandas as pd

# CSV読み込み
df_brands = pd.read_csv("csv/brand_20240422-2025-0412.csv", encoding="utf-8-sig")
df_final = pd.read_csv("csv/final_20240422-2025-0412.csv", encoding="utf-8-sig")

# 横方向（列方向）に結合
df_final = pd.concat([df_final, df_brands], axis=1)
df_final.to_csv("csv/final_20240422-2025-0412.csv", index=False, encoding="utf-8-sig")

In [ ]:
import pandas as pd

# CSV読み込み
df = pd.read_csv('csv/final_20240422-20250412.csv')

# 商品名列（適宜変更）
column_name = '商品名'

# カテゴリ判定関数
def label_category(text):
    if pd.isna(text):
        return ''
    text = str(text).lower()

    # カテゴリと対応するキーワード辞書
    category_keywords = {
        'Outer': ['コート', 'ジャケット', 'ブルゾン', 'アウター', 'ダウン', 'パーカー'],
        'Suit': ['スーツ', 'セットアップ', 'ジャケット＆パンツ', '上下セット'],
        'Tops': ['トップス', 'ブラウス', 'シャツ', 'ニット', 'カーディガン', 'カットソー', 'セーター', 'tシャツ'],
        'Bottoms': ['パンツ', 'スカート', 'ボトムス', 'ショートパンツ', 'ガウチョ'],
        'Dress': ['ワンピース', 'ドレス'],
        'Accessory': ['バッグ', '帽子', 'ハット', 'ベルト', 'スヌード', 'マフラー', 'ストール', '手袋', 'アクセサリー']
    }

    for category, keywords in category_keywords.items():
        if any(kw in text for kw in keywords):
            return category

    return ''  # 該当なし

# カテゴリラベル列の追加
df['カテゴリラベル'] = df[column_name].apply(label_category)

# 結果保存
df.to_csv('final_20240422-20250412.csv', index=False, encoding='utf-8-sig')

df

,注文日,商品名,請求金額,合計金額,個数,送料合計,ポイント利用額,クーポン利用総額,季節ラベル,ブランド名,カテゴリラベル
0,2024-04-26,美品【中古】ピエールカルダン pierre cardin お洒落ジャケット 表記Mサイズ(9...,23171,25600,1,0,429.0,2000,春夏,ピエールカルダン,Outer
1,2024-04-26,美品【中古】ポールスミス Poul Smith テーラードジャケット ジャケット 表記 38...,23171,25600,1,0,429.0,2000,春夏,ポールスミス,Outer
2,2024-04-26,新品同様【中古】ホコモモラ Jocomomola 緑 涼感 素敵ブラウス トップス 42号/...,9800,10800,1,0,0.0,1000,春夏,ホコモモラ,Tops
3,2024-04-27,美品【中古】組曲 KUMIKYOKU ワンピース 異素材ワンピース 大きいサイズ 表記 5号...,21900,21900,1,0,0.0,0,春夏,組曲,Dress
4,2024-05-02,美品【中古】ミラオーウェン mila owen タイトスカート ニットスカート スカート レ...,6300,16300,1,0,10000.0,0,春秋,ミラオーウェン,Tops
...,...,...,...,...,...,...,...,...,...,...,...
400,2025-03-27,美品【中古】ジブリーヌ Zibeline 東京イギン フォーマルジャケット ノーカラージャケ...,33500,33500,1,0,0.0,0,通年,ジブリーヌ,Outer
401,2025-03-29,新品同様【中古】スキャパ SCAPA フレアスカート 表記 38号 9号 M 相当 古着 紺...,8400,8900,1,0,500.0,0,春夏,スキャパ,Bottoms
402,2025-04-03,美品【中古】レオナール LEONARD デニムスカート 膝丈スカート スカート レディース ...,28000,28780,1,0,780.0,0,春秋,レオナール,Bottoms
403,2025-04-08,新品同様【中古】エプリス EPRISE NOIR3 東京ブラウス フォーマルブラウス ブラウ...,18000,20000,1,0,0.0,2000,通年,エプリス,Tops
